In [5]:
from singer.metrics import record_counter
import osmium
import shapely.wkb as wkblib
import singer
from typing import List

wkbfab = osmium.geom.WKBFactory()
wktfab = osmium.geom.WKTFactory()


class OsmHandler(osmium.SimpleHandler):
    def __init__(self, record_counter: record_counter):
        osmium.SimpleHandler.__init__(self)
        self.record_counter = record_counter

    def write_record(self, tags, wkt):
        name = tags.get('name', None)
        amenity = tags.get('amenity', None)

        singer.write_records('osm_poi', [{
            'name': name,
            'type': amenity,
            'geometry': wkt
        }])

        self.record_counter.increment()

    def node(self, n):
#         print(n.tags.get('shop', ''), 'amenity' in n.tags)
#         if 'amenity' in n.tags:
        if any(x in n.tags for x in ['amenity', 'shop']):
            wkt = wktfab.create_point(n)
            self.write_record(tags=n.tags, wkt=wkt)

    def area(self, a):
        if any(x in n.tags for x in ['amenity', 'shop']):
            wkb = wkbfab.create_multipolygon(a)
            poly = wkblib.loads(wkb, hex=True)
            centroid = poly.representative_point()
            wkt = centroid.wkt
            self.write_record(tags=a.tags, wkt=wkt)
        
            
with record_counter(log_interval=10) as counter:
    osm_handler = OsmHandler(record_counter=counter)
    osm_handler.apply_file('data/map.osm')

{"type": "RECORD", "stream": "osm_poi", "record": {"name": "Albert Heijn", "type": null, "geometry": "POINT(5.503666 51.4413015)"}}
{"type": "RECORD", "stream": "osm_poi", "record": {"name": "Action Eindhoven", "type": null, "geometry": "POINT(5.5033448 51.442243)"}}
{"type": "RECORD", "stream": "osm_poi", "record": {"name": null, "type": "post_box", "geometry": "POINT(5.5029273 51.4418341)"}}
{"type": "RECORD", "stream": "osm_poi", "record": {"name": "Dragon City", "type": "restaurant", "geometry": "POINT(5.504625 51.4417659)"}}
{"type": "RECORD", "stream": "osm_poi", "record": {"name": "Knabbel", "type": "pub", "geometry": "POINT(5.4997387 51.4419028)"}}
{"type": "RECORD", "stream": "osm_poi", "record": {"name": "De gulden", "type": "fast_food", "geometry": "POINT(5.5086572 51.4422422)"}}
{"type": "RECORD", "stream": "osm_poi", "record": {"name": "Ruud Schellekens Tweewielerspecialist", "type": null, "geometry": "POINT(5.5083874 51.4421255)"}}
{"type": "RECORD", "stream": "osm_poi", 

INFO METRIC: {"type": "counter", "metric": "record_count", "value": 37, "tags": {}}


NameError: name 'n' is not defined